In [1]:
#Basic Import
import pandas as pd
import numpy as np
import requests
import json
import keyboard
import os
import threading
import signal
import sys
import time
from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup
import re
#Import for Selenium
import logging

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException
search_url = 'https://danbolig.dk/soeg/?o=newest-desc'

#To Break while loops
is_active = True
list_of_links = []

# Create Selenium functions

**save_webriver_content:** save the 

**break_on_escape:** when escape is pressed, the scroller stops.

**selenium_scroller:** open Chrome winow, wait 15 seconds, scroll to bottom. Return links



In [2]:
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    filename='script_log.txt',
                    filemode='w')
def save_links(content):
    with open('list_of_links.txt', 'w') as file:
        file.write('\n'.join(content) + ',' + '\n')
        
def pause_program():
    global is_active
    is_active = False

def on_escape_press(event):
    if event.name == 'esc':
        pause_program()

keyboard.on_press(on_escape_press)

def get_links(driver):
    links = driver.find_elements(By.XPATH, "//a[starts-with(@href, '/bolig/')]")
    list_of_links = [link.get_attribute('href') for link in links]
    return list_of_links

def selenium_scroller(search_url):
    global is_active
    is_active = True
    driver = webdriver.Chrome()
    url = search_url
    driver.get(url)
    scroll_height = 0
    last_height = driver.execute_script("return document.body.scrollHeight")
    time.sleep(15)
    property_listings_div = driver.find_element(By.CSS_SELECTOR, 'div.t-search__list')
    #list_of_links = []
    while is_active:
        try:
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", property_listings_div)
            time.sleep(1)
            new_scroll_height = driver.execute_script("return arguments[0].scrollHeight;", property_listings_div)
            if new_scroll_height == scroll_height:
                break
            scroll_height = new_scroll_height
        except KeyboardInterrupt:
            pause_program()
            break  
        except WebDriverException as e:
            logging.error(f"Selenium error: {e}")
            driver.save_screenshot("screenshot.png")
        except Exception as e:
            logging.exception("An unexpected error occurred")
            driver.save_screenshot("error_screenshot.png")
        except WebDriverException as e:
            logging.error(f"Selenium error: {e}")
            driver.save_screenshot("screenshot.png")
            driver.quit()  # Close the driver
        except Exception as e:
            logging.exception("An unexpected error occurred")
            driver.save_screenshot("error_screenshot.png")
            driver.quit()  # Close the driver
    list_of_links = get_links(driver)
    # Collect links and content
    
    
#    driver.quit()  # Close the driver
    
    return list_of_links
 


## Get, save and print list of links
Press escape to break scrolling.

In [4]:
content = selenium_scroller(search_url)
save_links(content)
for i in content:
    print(i)

## Load the data from the links
Get zip code by regex, to separate roadname, zipcode and city in data


In [9]:
re_zip = r'https://danbolig\.dk/bolig/([^/]+)/(\d+)/'

zip_list = []
table_large = []
table_left = []
table_address = []
top_info_table = []
house_description = []
house_zip = []
house_data = []
def save_dataframe(frame, file_name):
    csv_filename = '{}.csv'.format(file_name)

    # Export the DataFrame to CSV
    frame.to_csv(csv_filename, index=False)
def get_page_content(url_list):
    for url in url_list:
        match = re.search(re_zip, url)
        if match:
            city = match.group(1)
            zip_code = match.group(2)
            zip_list.append(zip_code)
        try:
            print(url)
            # Send a request to the URL and get the content of the webpage
            response = requests.get(url, headers={'name': 'August Jensen', 'email': 'blz957@alumni.ku.dk', 'institution': 'University of Copenhagen'})
            soup = BeautifulSoup(response.content, 'lxml')

            # Access the large main table and add it to the list of large_tables
            info_table_large = soup.find_all('table')[0].find('tbody')
            table_large.append(info_table_large)

            # Access the small table to the left and add it to the list of left_tables
            info_table_small_left = soup.find_all('table')[1].find('tbody')
            table_left.append(info_table_small_left)

            # Access the small info table at the top (address) and add it to the list of top_addresses
            upper_info_table = soup.find_all('div', class_='o-propertyHero__info o-propertyHero__info--desktop')[0]

            # Find the addresses
            addresses = upper_info_table.find('h1')
            table_address.append(addresses)

            # Find house price and other stats
            info_table = upper_info_table.find_all('ul', class_='o-propertyHero__facts')[0]
            top_info_table.append(info_table)

            #Get house description
            description_tags = soup.find_all('div', class_='o-content o-propertyDescription__description a-paragraph')
            print("Number of description tags found:", len(description_tags)) 
            
            for description in description_tags:
                house_description.append(description.get_text(strip=True))
            if len(description_tags) == 0:
                house_description.append('No description available')
            time.sleep(1)
        except IndexError:
            print("IndexError: Skipping this link.")
            continue



def pass_content_to_dataframe():

    for house in range(len(table_large)):
        main_table_dict = {}

        main_table = table_large[house].find_all('td')
        for i in range(0, len(main_table), 2):
            name = main_table[i].text
            value = main_table[i + 1].text
            value = re.sub(r'\s*(m²|kr\.)', '', value)
            main_table_dict[name] = value

        top_table = top_info_table[house].find_all('span')
        for i in range(0, len(top_table), 2):
            name = top_table[i].text
            value = top_table[i + 1].text
            value = re.sub(r'\s*(m²|kr\.)', '', value)
            if name != 'Energimærke':
                main_table_dict[name] = value.strip()

        name = "address"
        value = ' '.join(table_address[house].text.split())
        main_table_dict[name] = value

        # Add description and zip_code fields from the provided lists
        main_table_dict['description'] = house_description[house]
        main_table_dict['zip code'] = zip_list[house]

        house_data.append(main_table_dict)

    df = pd.DataFrame(house_data)
    # Assuming there's a function save_dataframe that saves the DataFrame to a file
    save_dataframe(df, 'content')
    return df

def get_correct_address(row):
    address_parts = row['address'].split()
    
    # Find the index of the zip code in the address parts
    zip_index = address_parts.index(row['zip code'])
    
    # Join the parts to the left of the zip code as the road
    road = ' '.join(address_parts[:zip_index])
    
    # Join the parts to the right of the zip code as the city
    city = ' '.join(address_parts[zip_index + 1:])
    
    return pd.Series({'City': city, 'zip code': row['zip code'], 'Road': road})

## Get information


In [10]:
get_page_content(content)
df = pass_content_to_dataframe()

https://danbolig.dk/bolig/vejle/7100/villa/0950001202-095/
Number of description tags found: 1
https://danbolig.dk/bolig/kolding/6000/lejlighed/0170000991-017/
Number of description tags found: 1
https://danbolig.dk/bolig/aarhus/8200/lejlighed/2730000448-273/
Number of description tags found: 1
https://danbolig.dk/bolig/kerteminde/5290/villa/2590001307-259/
Number of description tags found: 1
https://danbolig.dk/bolig/vordingborg/4760/villa/2040000675-204/
Number of description tags found: 1
https://danbolig.dk/bolig/mariagerfjord/9500/villa/4750000371-475/
Number of description tags found: 1
https://danbolig.dk/bolig/halsnaes/3390/villa/2930000213-293/
Number of description tags found: 1
https://danbolig.dk/bolig/vesthimmerlands/9600/villa/2580000393-258/
Number of description tags found: 1
https://danbolig.dk/bolig/odense/5260/villa/2590001283-259/
Number of description tags found: 1
https://danbolig.dk/bolig/ishoej/2635/villa/2760000207-276/
Number of description tags found: 1
https

In [7]:
response = requests.get('https://danbolig.dk/bolig/vejle/7182/villa/0950001336-095/')
soup = BeautifulSoup(response.content, 'html.parser')
print(soup)



<!DOCTYPE html>

<html lang="da">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>Kærbøllinghusevej 31 - Villa til Salg - danbolig</title>
<meta content="telephone=no" name="format-detection">
<meta content="Villa til Salg på Kærbøllinghusevej 31. Se boligens billeder, plantegning, salgsopstilling, rapporter og nabolag hos danbolig" name="description">
<meta content="Kærbøllinghusevej 31 - Villa til Salg - danbolig" property="og:title"/>
<meta content="Villa til Salg på Kærbøllinghusevej 31. Se boligens billeder, plantegning, salgsopstilling, rapporter og nabolag hos danbolig" property="og:description"/>
<meta content="https://danbolig.mindworking.eu/api/mediaData/mediaDataMediaPurposeId/SU1lZGlhRGF0YTp7Ik1lZGlhRGF0YUlkIjo1NzI3OTAwLCJNZWRpYVB1cnBvc2VJZCI6MjJ9/imageHash/89fd00cfd3c986cda3f833f520694cc7/imageSize/assets/inline/true?deviceid=DB_mnvxs43mnbn23&amp;width=750&amp;height=500" property="og:image"/>
<meta content="750"